In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
print(os.listdir("../input"))
import shutil

['train_val_list.txt', 'images_007', 'BBox_List_2017.csv', 'images_011', 'LOG_CHESTXRAY.pdf', 'FAQ_CHESTXRAY.pdf', 'images_008', 'images_006', 'ARXIV_V5_CHESTXRAY.pdf', 'images_012', 'images_004', 'test_list.txt', 'Data_Entry_2017.csv', 'images_009', 'images_002', 'images_003', 'images_001', 'README_CHESTXRAY.pdf', 'images_005', 'images_010']


In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from fastai import *
from fastai.vision import *

In [3]:
df = pd.read_csv('../input/Data_Entry_2017.csv')

    ## Uncomment

In [4]:
lst_files_001 = Path('../input/images_001/images').ls()
lst_files_002 = Path('../input/images_001/images').ls()
lst_files_003 = Path('../input/images_003/images').ls()
lst_files_004 = Path('../input/images_003/images').ls()
lst_files_005 = Path('../input/images_005/images').ls()

lst_files = lst_files_001 + lst_files_002 + lst_files_003 + lst_files_004 + lst_files_005

In [5]:
# Filtering the dataset for later use.
lst_names = list(map(os.path.basename, lst_files))
df_filtered = df[df['Image Index'].isin(lst_names)] # Filtered with the files which we are interested in.

In [6]:
def get_label(fname):
    file_name = (Path(os.path.basename(fname)))
    df_find = df.loc[df['Image Index'] == str(file_name)]
    labelcol = 'Finding Labels'
    retval = df_find[labelcol].values
    retval = ' '.join(map(str, retval))
    #print((retval))
    return((retval))

## Uncomment the following blocks for full training

In [7]:
il = ImageList.from_folder(path =  Path('../input/images_001/images'))
il = il.add(ImageList.from_folder(path =  Path('../input/images_002/images')))
il = il.add(ImageList.from_folder(path =  Path('../input/images_003/images')))
il = il.add(ImageList.from_folder(path =  Path('../input/images_004/images')))
il = il.add(ImageList.from_folder(path =  Path('../input/images_005/images')))

In [8]:
ll  = list(map(get_label, Path('../input/images_001/images').ls())) 
ll = ll + list(map(get_label, Path('../input/images_002/images').ls()))
ll = ll + list(map(get_label, Path('../input/images_003/images').ls()))
ll = ll + list(map(get_label, Path('../input/images_004/images').ls()))
ll = ll + list(map(get_label, Path('../input/images_005/images').ls()))

In [9]:
sd = il.split_by_rand_pct(0.1) # split data
ll = sd.label_from_func(get_label, label_delim = "|")
data = ll.transform(size=128).databunch(bs = 16).normalize(imagenet_stats)

In [10]:
data

ImageDataBunch;

Train: LabelList (40500 items)
x: ImageList
Image (3, 128, 128),Image (3, 128, 128),Image (3, 128, 128),Image (3, 128, 128),Image (3, 128, 128)
y: MultiCategoryList
Atelectasis;Infiltration,No Finding,No Finding,No Finding,No Finding
Path: ../input/images_001/images;

Valid: LabelList (4499 items)
x: ImageList
Image (3, 128, 128),Image (3, 128, 128),Image (3, 128, 128),Image (3, 128, 128),Image (3, 128, 128)
y: MultiCategoryList
No Finding,Emphysema;Fibrosis;Pneumothorax,Pleural_Thickening,No Finding,Consolidation
Path: ../input/images_001/images;

Test: None

In [11]:
#valid_filenames = data.valid_ds.items
#len(valid_filenames)

In [12]:
print(data.classes)

['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Effusion', 'Emphysema', 'Fibrosis', 'Hernia', 'Infiltration', 'Mass', 'No Finding', 'Nodule', 'Pleural_Thickening', 'Pneumonia', 'Pneumothorax']


In [13]:
#data.show_batch(rows=3, figsize=(15,15))

In [14]:
acc_02 = partial(accuracy_thresh, thresh=0.2)
f_02 = partial(fbeta, thresh=0.2)
acc_03 = partial(accuracy_thresh, thresh=0.3)
f_03 = partial(fbeta, thresh=0.3)
acc_04 = partial(accuracy_thresh, thresh=0.4)
f_04 = partial(fbeta, thresh=0.4)
acc_05 = partial(accuracy_thresh, thresh=0.5)
f_05 = partial(fbeta, thresh=0.5)

In [15]:
#p_thresh = .5

#def accuracy(input:Tensor, targs:Tensor)->Rank0Tensor:
    #"Computes accuracy with `targs` when `input` is bs * n_classes."
    #n = targs.shape[0]
   # input = input.argmax(dim=-1).view(n,-1)
    #targs = targs.view(-1).long()
    #return (input==targs).float().mean()

In [16]:
#arch = 
#acc_02 = partial(accuracy_thresh, thresh=p_thresh)
#f_score = partial(fbeta, thresh=p_thresh)
learn = cnn_learner(data, models.resnet50, metrics=[acc_02, f_02, acc_03, f_03, acc_04, f_04, acc_05, f_05],  model_dir="/tmp/model/")

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /tmp/.cache/torch/checkpoints/resnet50-19c8e357.pth
100%|██████████| 97.8M/97.8M [00:04<00:00, 22.6MB/s]


In [17]:
#learn.lr_find()
#learn.recorder.plot()

In [18]:
import time
start_time = time.time()

In [19]:
learn.fit_one_cycle(5, 2 * 1e-2)

epoch,train_loss,valid_loss,accuracy_thresh,fbeta,accuracy_thresh,fbeta,accuracy_thresh,fbeta,accuracy_thresh,fbeta,time
0,0.207936,0.204403,0.926162,0.577443,0.929333,0.578536,0.929362,0.578573,0.929511,0.577684,20:02


In [20]:
end_time = time.time()
print("Training time:", end_time - start_time, "seconds")

Training time: 5872.515463352203 seconds


In [21]:
(learn.summary())

Sequential
Layer (type)         Output Shape         Param #    Trainable 
Conv2d               [64, 64, 64]         9,408      False     
______________________________________________________________________
BatchNorm2d          [64, 64, 64]         128        True      
______________________________________________________________________
ReLU                 [64, 64, 64]         0          False     
______________________________________________________________________
MaxPool2d            [64, 32, 32]         0          False     
______________________________________________________________________
Conv2d               [64, 32, 32]         4,096      False     
______________________________________________________________________
BatchNorm2d          [64, 32, 32]         128        True      
______________________________________________________________________
Conv2d               [64, 32, 32]         36,864     False     
___________________________________________________

In [22]:
preds, _ = learn.get_preds(ds_type=DatasetType.Valid)

In [23]:
def get_labelled_preds(preds, p_thresh):
    labelled_preds = [';'.join([learn.data.classes[i] for i,p in enumerate(pred) if p > p_thresh]) for pred in preds]
    cleaned_labelled_preds = []
    for l in labelled_preds:
        lx = l
        if (l != "No Finding"):
            lx = l.replace("No Finding;", "")
        if (l != "No Finding"):
            lx = lx.replace("No Finding", "")
        cleaned_labelled_preds.append(lx)
    #actual = list(learn.data.valid_ds.y)
    #act = []
    #for a in actual:
    #     act.append(str(a).replace('Multicategory ', ''))
    return(cleaned_labelled_preds, labelled_preds)

In [24]:
# Cleaning of Labels happening here.
#labelled_preds = [';'.join([learn.data.classes[i] for i,p in enumerate(pred) if p > p_thresh]) for pred in preds]
#cleaned_labelled_preds = []
#for l in labelled_preds:
    #lx = l
    #if (l != "No Finding"):
     #   lx = l.replace("No Finding;", "")
    #if (l != "No Finding"):
      #  lx = lx.replace("No Finding", "")
    #cleaned_labelled_preds.append(lx)

#actual = list(learn.data.valid_ds.y)
#act = []
#for a in actual:
     #act.append(str(a).replace('Multicategory ', ''))

In [25]:
y_actuals = []
for i in data.valid_ds.y:
    y_actuals.append(str(i).replace('Multicategory ', ''))

In [26]:
df_y = pd.DataFrame({'actual': y_actuals})
df_y

,actual
0,No Finding
1,Emphysema;Fibrosis;Pneumothorax
2,Pleural_Thickening
3,No Finding
4,Consolidation
5,No Finding
6,Pneumothorax
7,Atelectasis
8,No Finding
9,No Finding


In [27]:
cleaned_02, predicted_02 = get_labelled_preds(preds, p_thresh = 0.2)
cleaned_03, predicted_03 = get_labelled_preds(preds, p_thresh = 0.3)
cleaned_04, predicted_04 = get_labelled_preds(preds, p_thresh = 0.4)
cleaned_05, predicted_05 = get_labelled_preds(preds, p_thresh = 0.5)

In [28]:
#df_1 = pd.DataFrame({'actual':act, 'predicted':labelled_preds,  'cleaned':cleaned_labelled_preds}, columns=['actual', 'predicted','cleaned'])


In [29]:
df_1 = pd.DataFrame({'cleaned_02':cleaned_02, 'cleaned_03':cleaned_03,  'predicted_02':predicted_02, 'predicted_03':predicted_03}, columns=['cleaned_02', 'cleaned_03','predicted_02', 'predicted_03'])


In [30]:
df_1a = pd.DataFrame({'cleaned_04':cleaned_04, 'cleaned_05':cleaned_05,  'predicted_04':predicted_04, 'predicted_05':predicted_05}, columns=['cleaned_04', 'cleaned_05','predicted_04', 'predicted_05'])


In [31]:
arr = preds.data.cpu().numpy()
#arr.shape
df_2 = pd.DataFrame(arr)
#Assigning the column names to classes.
df_2.columns = learn.data.classes
df_results = pd.concat([df_y,df_1,df_1a, df_2],axis = 1); df_results
df_results.to_csv('error_analysis.csv', index=False)
df_results

/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

,actual,cleaned_02,cleaned_03,predicted_02,predicted_03,cleaned_04,cleaned_05,predicted_04,predicted_05,Atelectasis,...,Emphysema,Fibrosis,Hernia,Infiltration,Mass,No Finding,Nodule,Pleural_Thickening,Pneumonia,Pneumothorax
0,No Finding,No Finding,No Finding,No Finding,No Finding,No Finding,No Finding,No Finding,No Finding,3.214205e-02,...,2.637776e-02,0.032551,1.688644e-03,0.105655,4.877423e-02,0.611390,6.463388e-02,3.845958e-02,5.754460e-03,8.178507e-02
1,Emphysema;Fibrosis;Pneumothorax,Effusion;Infiltration;,No Finding,Effusion;Infiltration;No Finding,No Finding,,,,,7.113861e-02,...,6.186417e-02,0.088757,3.943702e-03,0.210037,1.170581e-01,0.308413,1.194766e-01,1.387761e-01,2.191644e-02,1.877612e-01
2,Pleural_Thickening,Effusion;Infiltration;,,Effusion;Infiltration;No Finding,,,,,,8.748181e-02,...,5.917745e-02,0.051249,4.322996e-03,0.255908,1.066288e-01,0.299902,1.149578e-01,1.025826e-01,2.130810e-02,1.355409e-01
3,No Finding,No Finding,No Finding,No Finding,No Finding,No Finding,No Finding,No Finding,No Finding,4.309194e-02,...,9.473571e-03,0.024848,1.870806e-03,0.073730,3.388739e-02,0.684333,6.158843e-02,2.365407e-02,7.288698e-03,1.974599e-02
4,Consolidation,Infiltration;,No Finding,Infiltration;No Finding,No Finding,No Finding,,No Finding,,1.309365e-01,...,3.381859e-02,0.014116,2.824323e-03,0.219146,6.886930e-02,0.433515,5.644194e-02,3.251100e-02,1.708228e-02,5.172991e-02
5,No Finding,No Finding,No Finding,No Finding,No Finding,No Finding,No Finding,No Finding,No Finding,1.179007e-01,...,1.127983e-02,0.003617,1.091793e-03,0.123097,2.571272e-02,0.701855,2.159663e-02,7.207587e-03,6.095644e-03,1.372761e-02
6,Pneumothorax,Effusion;Infiltration;,Effusion,Effusion;Infiltration;No Finding,Effusion,Effusion,,Effusion,,1.618381e-01,...,4.448631e-02,0.048458,4.623175e-03,0.218755,9.799411e-02,0.246950,8.873859e-02,1.127710e-01,1.912570e-02,1.334438e-01
7,Atelectasis,Atelectasis;Effusion;,Atelectasis;,Atelectasis;Effusion;No Finding,Atelectasis;No Finding,No Finding,,No Finding,,3.295724e-01,...,1.046434e-02,0.005682,1.956574e-03,0.185579,2.351731e-02,0.403990,2.087054e-02,1.484331e-02,1.056635e-02,2.452150e-02
8,No Finding,No Finding,No Finding,No Finding,No Finding,No Finding,No Finding,No Finding,No Finding,2.578264e-02,...,4.698606e-03,0.008611,7.954968e-04,0.054423,1.781996e-02,0.830068,2.551785e-02,9.643994e-03,2.756933e-03,1.230576e-02
9,No Finding,No Finding,No Finding,No Finding,No Finding,No Finding,No Finding,No Finding,No Finding,3.849203e-02,...,1.393282e-02,0.022962,1.713239e-03,0.087150,3.047659e-02,0.706151,4.023891e-02,2.162666e-02,5.609983e-03,3.775079e-02


### EXPORTING MODEL AND OTHER INFORMATION

In [32]:
import os
import io
import tarfile
import PIL
import boto3
from fastai.vision import *

IMAGE_SIZE = 128 # Should match with the size specified while building the model.
learn.save("chestxray-model")
source_file = learn.model_dir + "chestxray-model" + ".pth"
dest_file = "chestxray-model" + ".pth"
shutil.copy2(source_file, dest_file)
save_texts('classes.txt', data.classes)
trace_input = torch.ones(1,3,IMAGE_SIZE,IMAGE_SIZE).cuda()
jit_model = torch.jit.trace(learn.model.float(), trace_input)
model_file='chestxray_resnet50_jit.pth'
output_path = str({model_file})
#torch.jit.save(jit_model, output_path)
torch.jit.save(jit_model, 'chestxray_resnet50_jit.pth')
tar_file='chestxray.tar.gz'
classes_file='classes.txt'

print(os.listdir("."))
with tarfile.open(tar_file, 'w:gz') as f:
    f.add(model_file, arcname=model_file)
    f.add(classes_file, arcname=classes_file)

['chestxray_resnet50_jit.pth', '__output__.json', '__notebook__.ipynb', 'classes.txt', 'chestxray-model.pth', 'error_analysis.csv']
